### Installation

In [ ]:
#pip install bertopic

In [ ]:
#pip install matplotlib

In [ ]:
#pip install protobuf # für NLP/swissbert

In [97]:
import pickle
import plotly.express as px
import plotly.io as pio
import matplotlib as plt
import os
from bertopic.representation import PartOfSpeech
from sklearn.feature_extraction.text import CountVectorizer

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.backend import BaseEmbedder
import pandas as pd

### Komplettes Korpus

In [98]:
# Englishcsv_file_path = '/Users/davideventre/Desktop/telegram_daten/subcorpus_tp1.csv'
# Read the CSV file
# wort = input("Enter the name of the variable: ")
# Name of the output folder and the file
teilprojekt = 'tp1'
output_folder_name = "tp1_solidaritaet"
# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_name):
    os.makedirs(output_folder_name)
# Get the current working directory
current_working_directory = os.getcwd()
filename = "/Users/davideventre/Desktop/telegram_daten/sampled_subc_tp1.csv"
csv_file_path = os.path.join(current_working_directory, filename)
df = pd.read_csv(csv_file_path, sep='\t', low_memory=False)
# Get the number of rows in the DataFrame
num_rows = len(df)
korpus_content = df['text_content'].astype(str).tolist()
korpus_text_date = df['text_date'].astype(str).tolist()
korpus_text_source = df['text_source'].astype(str).tolist()
korpus_text_id = df['text_id'].astype(str).tolist()

### Zeitraum

In [99]:
import pandas as pd
from datetime import datetime

# Function to parse dates
def parse_date(date_str):
    for fmt in ('%d-%m-%Y', '%Y-%m'):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            pass
    raise ValueError('no valid date format found')

# Define your time frames as a dictionary with corpus names as keys
time_frames = {
    'korpus1': (parse_date('01-01-2020'), parse_date('01-01-2021')),
}

# Initialize your sentences_corpus dictionary
korpus_content = []
korpus_text_date = []
korpus_text_source = []
korpus_text_id = []

# Specify the path to your CSV file
csv_file_path = "/Users/davideventre/Desktop/telegram_daten/sampled_subc_tp1.csv"  # Update with the actual file path

# Load your DataFrame
df = pd.read_csv(csv_file_path, sep='\t')  # Replace with your actual file path

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    try:
        channel_date = parse_date(row['text_date'])
    except ValueError:
        continue  # Skip rows with invalid date formats

    # Check each corpus defined in time_frames
    for corpus_name, (start_date, end_date) in time_frames.items():
        if start_date <= channel_date <= end_date:
            korpus_content.append(row['text_content'])
            korpus_text_date.append(row['text_date'])
            korpus_text_source.append(row['text_source'])
            korpus_text_id.append(row['text_id'])

print(len(korpus_content))
print(len(korpus_text_date))
print(len(korpus_text_source))
print(len(korpus_text_id))

721
721
721
721


### Quelle

In [100]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = "/Users/davideventre/Desktop/telegram_daten/sampled_subc_tp1.csv"  # Update with the actual file path

# Load the CSV file using pandas
df = pd.read_csv(csv_file_path, sep='\t')

# Define search words for each corpus as a list
corpora = ['faz', 'BILD']  # Add as many as needed

# Initialize your sentences_corpus dictionary
korpus_content = []
korpus_text_date = []
korpus_text_source = []
korpus_text_id = []

# Specify the path to your CSV file

# Load your DataFrame
df = pd.read_csv(csv_file_path, sep='\t')  # Replace with your actual file path

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    channel_id = row['text_source']
    message = row['text_content'] 
    for corpus in corpora:
        if corpus in channel_id:
            korpus_content.append(row['text_content'])
            korpus_text_date.append(row['text_date'])
            korpus_text_source.append(row['text_source'])
            korpus_text_id.append(row['text_id'])
            
print(len(korpus_content))
print(len(korpus_text_date))
print(len(korpus_text_source))
print(len(korpus_text_id))

23136
23136
23136
23136


### BERTopic

In [101]:
from transformers.pipelines import pipeline
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
#embedding_model = pipeline("feature-extraction", model="bert-base-german-cased")
#embedding_model = pipeline(model="ZurichNLP/swissbert")
#embedding_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

In [102]:
from umap import UMAP
from sklearn.decomposition import PCA
# n_neighbors: höhere Werte nehmen eine "globalere" Perspektive der Embeddings ein (grössere Cluster)
# n_neighbors: tiefere Werte nehmen eine "lokalere" Perspektive der Embeddings ein
# n_components: tiefere Werte beeinflussen die Qualität der Embeddings
# n_components: hohe Werte dauern länger und HDBScan braucht länger für die Berechnung

dim_model = UMAP(n_neighbors=15, n_components=10, min_dist=0.0, metric='cosine')


# Eine schnellere Art die Dimensionen zu reduzieren
#dim_model = PCA(n_components=5)
#topic_model = BERTopic(umap_model=dim_model)

In [103]:
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans

# min_cluster_size: wie gross die Cluster mindestens sein müssen
# min_samples: die Zahl der Outlier, tiefere Zahlen reduzieren die Outlier
cluster_model = HDBSCAN(min_cluster_size=25, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=3)

# andere Art des Clustering, das keine Ausreisser produziert
#cluster_model = KMeans(n_clusters=50)

In [104]:
from sklearn.feature_extraction.text import CountVectorizer
# min_df: wie oft ein Wort vorkommen muss, bevor es in die Repräsentation gelangt
# so kann man bei grossen Dokumenten die Berechnung verkürzen
# ngram_range: bestimmt die Länge der Ngrams, die in der Repräsentation erscheinen
vectorizer_model = CountVectorizer(min_df=1, ngram_range=(1, 3))

# max_features: topic term matrix wird kontrolliert. anstelle das man min_df einstellen muss
#vectorizer_model = CountVectorizer(max_features=10_000)

In [105]:
from bertopic.vectorizers import ClassTfidfTransformer
# man kann folgende Parameter einfügen: 
# reduce_frequent_words oder BM25
ctfidf_model = ClassTfidfTransformer()
# besser mit Stoppwörtern:
#ctfidf_model = ClassTfidfTransformer(bm25_weighting=True)
#ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

In [106]:
# verschiedene Repräsentationsmöglichkeiten:
# Keywords
representation_model = [KeyBERTInspired(top_n_words=10), MaximalMarginalRelevance(diversity=0.95)]
# Sprachliche Muster
#pos_patterns = [[{'POS': 'ADJ'}, {'POS': 'NOUN'}], [{'POS': 'NOUN'}], [{'POS': 'ADJ'}]]
#representation_model = PartOfSpeech("de_core_web_sm", pos_patterns=pos_patterns)
# möglichst unterschiedliche Wörter
#representation_model = MaximalMarginalRelevance(diversity=0.3)

### Ausführung BERTopic

In [107]:
topic_model = BERTopic(embedding_model=embedding_model, representation_model=representation_model, umap_model=dim_model, hdbscan_model=cluster_model, vectorizer_model=vectorizer_model, ctfidf_model=ctfidf_model)
topic_model.fit(korpus_content)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
To disable this warning, you can either:
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment varia

In [108]:
topics, probs = topic_model.fit_transform(korpus_content)
topic_model_distr = BERTopic().fit(korpus_content)
# Reduce outliers
new_topics = topic_model.reduce_outliers(korpus_content, topics)
# versions of dependencies and python used. loading and saving model => same dependencies and python
# saved in one version of Bertopic should not be loaded in others
#topic_model = BERTopic.load("meditopic_model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

### Übersicht Daten

In [109]:
def visualize_and_save_topics_html(model, output_folder_name, m, text_content, text_id, text_date, text_source):
    meta_data_id = {}
    meta_data_id['text_id'] = text_id
    meta_data_id['text_date'] = text_date
    meta_data_id['text_source'] = text_source
    topic_info = model.get_document_info(text_content, metadata=meta_data_id)
    # Assuming topic_info is a list of dictionaries with keys like 'document_id', 'topic_distribution', etc.
    # Saving the data to a .txt file
    topic_data = pd.DataFrame(topic_info)
    csv_file_path1 = os.path.join(output_folder_name, m + 'list_' + '.csv')
    topic_data.to_csv(csv_file_path1, index=False)  # Set index=False to exclude row indices in the output

visualize_and_save_topics_html(topic_model, output_folder_name, teilprojekt, korpus_content, korpus_text_id, korpus_text_date, korpus_text_source)#, korpus_embeddings)

### Topics

In [110]:
def visualize_and_save_topics_html(model, output_folder_name, m):
    fig = model.visualize_topics()
    html = pio.to_html(fig)
    html_file_path = os.path.join(output_folder_name, m + '_topic' + '.html')
    with open(html_file_path, 'w') as f:
        f.write(html)
visualize_and_save_topics_html(topic_model, output_folder_name, teilprojekt)

### Barchart

In [111]:
def visualize_and_save_barchart_topics(model, output_folder_name, m):
    fig0 = model.visualize_barchart(top_n_topics=20, n_words=20)
    # Convert the figure to HTML
    html0 = pio.to_html(fig0)
    html_file_path0 = os.path.join(output_folder_name, m + '_barchart' + '.html')
    with open(html_file_path0, 'w') as f:
        f.write(html0)
    

visualize_and_save_barchart_topics(topic_model, output_folder_name, teilprojekt)

### Heatmap

In [112]:
# Function to visualize and save heatmap
def visualize_and_save_heatmap(model, output_folder_name, m):
    heatmap = model.visualize_heatmap()
    html4 = pio.to_html(heatmap)
    html_file_path3 = os.path.join(output_folder_name, m + '_heatmap' + '.html')
    with open(html_file_path3, 'w') as f:
        f.write(html4)

visualize_and_save_heatmap(topic_model, output_folder_name, teilprojekt)

### Hierarchy

In [113]:
def visualize_and_save_hierarchical_topics(model, output_folder_name, m, korpus_content):
    hierarchical_topics = model.hierarchical_topics(korpus_content)
    fig2 = model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)
    html2 = pio.to_html(fig2)
    html_file_path2 = os.path.join(output_folder_name, m + '_hierarchical' + '.html')
    with open(html_file_path2, 'w') as f:
        f.write(html2)
    
visualize_and_save_hierarchical_topics(topic_model, output_folder_name, teilprojekt, korpus_content)

100%|██████████| 125/125 [49:23<00:00, 23.71s/it]


### Dynamic

In [114]:
def visualize_clusters(model, output_folder_name, m, korpus_content, text_date):
    try:
        topics_over_time = model.topics_over_time(korpus_content, text_date, nr_bins=10)
        fig1 = model.visualize_topics_over_time(topics_over_time, top_n_topics=100)
        html1 = pio.to_html(fig1)
        html_file_path1 = os.path.join(output_folder_name, m + 'dynamic' + '.html')
        with open(html_file_path1, 'w') as f:
            f.write(html1)
    except ValueError:
        print('ValueError')
        pass
visualize_clusters(topic_model, output_folder_name, teilprojekt, korpus_content, korpus_text_date)

ValueError


### Document Clusters

In [115]:
korpus_embeddings = embedding_model.encode(korpus_content, show_progress_bar=True)
def visualize_clusters(model, output_folder_name, m, korpus_content, embeddings):
    try:
        visualize_docs = model.visualize_documents(korpus_content, embeddings=embeddings, hide_annotations=True, hide_document_hover=True, width=2400, height=1400)
        html_file_path7 = os.path.join(output_folder_name, m + 'documents' + '.html')
        html3 = pio.to_html(visualize_docs)
        with open(html_file_path7, 'w') as f:
            f.write(html3)
    except ValueError:
        print('ValueError')
        pass
    # Get document info
    
visualize_clusters(topic_model, output_folder_name, teilprojekt, korpus_content, korpus_embeddings)

Batches: 100%|██████████| 723/723 [12:40<00:00,  1.05s/it]


### Class

In [116]:
def visualize_clusters(model, output_folder_name, m, korpus_content, classes):
    # Topics ordered by a specific class 
    try:
        topics_per_class = model.topics_per_class(korpus_content, classes=classes)
        visualize_class = model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
        html5 = pio.to_html(visualize_class)
        html_file_path5 = os.path.join(output_folder_name, m + 'class' + '.html')
        with open(html_file_path5, 'w') as f:
            f.write(html5)
    except ValueError:
        print(f'inconsistent shapes {m}')
visualize_clusters(topic_model, output_folder_name, teilprojekt, korpus_content, korpus_text_source)

### Distribution of Topics in Topic

In [117]:
def visualize_clusters(model, output_folder_name, m, text_content, classes):
    # Topics ordered by a specific class 
    try:
        topic_distr, _ = model.approximate_distribution(text_content)
        visualize_class = model.visualize_distribution(topic_distr[0])
        html5 = pio.to_html(visualize_class)
        html_file_path5 = os.path.join(output_folder_name, 'distribution' + '.html')
        with open(html_file_path5, 'w') as f:
            f.write(html5)
    except ValueError:
        print(f'inconsistent shapes {m}')
visualize_clusters(topic_model, output_folder_name, "korpus", korpus_content, korpus_text_source)#, korpus1_embeddings)


NameError: name 'm' is not defined